In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-data/Numeric_data.csv
/kaggle/input/twitter-data/Text_data.csv


In [2]:
import pandas as pd


In [3]:
numeric_data=pd.read_csv('/kaggle/input/twitter-data/Numeric_data.csv')
text_data=pd.read_csv('/kaggle/input/twitter-data/Text_data.csv')

In [4]:
numeric_data.head()


,word_counts,character_counts,avg_word_len,stop_word_len,mentions,hash_tags,numeric_counts,upper_counts,emails_counts,urls_counts
0,6,33,4.500000,2,1,0,0,0,0,0
1,8,62,6.625000,2,1,0,0,1,0,0
2,14,56,3.000000,10,0,0,0,0,0,0
3,15,80,4.333333,5,0,0,0,0,0,0
4,20,131,5.500000,5,1,0,0,0,0,0


In [5]:
text_data=text_data.dropna(axis=0)

In [6]:
text_data=text_data[:20000]

In [7]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## On Numeric_data

In [24]:
cv=CountVectorizer()
text_counts=cv.fit_transform(text_data['text'])
text_counts.toarray().shape
data_bog=pd.DataFrame(text_counts.toarray(),columns=cv.get_feature_names())
X=data_bog
Y=text_data['target']
data_bog.shape

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(20000, 30189)

In [25]:
sgd   = SGDClassifier(n_jobs=-1,random_state=42,max_iter=1000)
lgr   = LogisticRegression(random_state=42,max_iter=1000)
lgrcv = LogisticRegressionCV(cv=2,random_state=42,max_iter=1000)
svm   = LinearSVC(random_state=42,max_iter=1000)
rfc   = RandomForestClassifier(random_state=42,n_jobs=-1,n_estimators=500)
clf={'SGD':sgd,'LGR':lgr,'LGRCV':lgrcv,'SVM':svm,'RFC':rfc}
clf

{'SGD': SGDClassifier(n_jobs=-1, random_state=42),
 'LGR': LogisticRegression(max_iter=1000, random_state=42),
 'LGRCV': LogisticRegressionCV(cv=2, max_iter=1000, random_state=42),
 'SVM': LinearSVC(random_state=42),
 'RFC': RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)}

In [26]:
def get_max_ac(d):
  return max(d, key = d.get)
def classify(x,y):
    accuracy_dict={}
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100,stratify=y)
    for key in clf.keys():
         print(f'\nProcessing : {key}-->\n')
         clf[key].fit(x_train,y_train)
         y_pred=clf[key].predict(x_test)
         print('\t',key,' ---> ',accuracy_score(y_test,y_pred))
         accuracy_dict[key]=accuracy_score(y_test,y_pred)
    
    return get_max_ac(accuracy_dict)

In [27]:
best_model=classify(X,Y)


Processing : SGD-->

	 SGD  --->  0.9995

Processing : LGR-->

	 LGR  --->  0.9995

Processing : LGRCV-->

	 LGRCV  --->  0.9995

Processing : SVM-->

	 SVM  --->  0.99925

Processing : RFC-->

	 RFC  --->  0.9995


In [28]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
model=clf[best_model]
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print('Accuracy:',accuracy_score(y_pred,y_test))

Accuracy: 0.99925


In [29]:
import pickle
pickle.dump((model,cv), open('best_model.pkl', 'wb'))

In [39]:
md,cv1 = pickle.load(open('/kaggle/working/best_model.pkl', 'rb'))

In [47]:
text=['what are you trying to tell me that i am what']
text_counts=cv1.transform(text)
text_counts.shape

(1, 30189)

In [48]:
md.predict(text_counts)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([0])